In [61]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e6/sample_submission.csv
/kaggle/input/playground-series-s5e6/train.csv
/kaggle/input/playground-series-s5e6/test.csv


In [49]:
# libraries to download
import pandas as pd
# Data manipulation and analysis
import pandas as pd
import numpy as np

# Visualization (optional, but helpful for EDA)
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Machine learning models
from sklearn.ensemble import RandomForestClassifier

# Model selection and evaluation
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV

# Utility for saving/loading models and encoders
import joblib

# For ignoring warnings (optional)
import warnings
warnings.filterwarnings('ignore')


In [50]:
df = pd.read_csv("/kaggle/input/playground-series-s5e6/train.csv")
df.head()

,id,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,0,37,70,36,Clayey,Sugarcane,36,4,5,28-28
1,1,27,69,65,Sandy,Millets,30,6,18,28-28
2,2,29,63,32,Sandy,Millets,24,12,16,17-17-17
3,3,35,62,54,Sandy,Barley,39,12,4,10-26-26
4,4,35,58,43,Red,Paddy,37,2,16,DAP


### data exploration

In [51]:
import pandas as pd

# Load the datasets using Kaggle input paths
train_df = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')
sample_submission_df = pd.read_csv('/kaggle/input/playground-series-s5e6/sample_submission.csv')

# --- TRAINING DATA INSPECTION ---
print("\n=== TRAINING DATA ===\n")
print("--- Info ---")
train_df.info()
print("\n--- Description (Numerical Columns) ---")
print(train_df.describe())
print("\n--- Missing Values ---")
print(train_df.isnull().sum())
print("\n--- Duplicate Rows ---")
print(train_df.duplicated().sum())

# --- TEST DATA INSPECTION ---
print("\n=== TEST DATA ===\n")
print("--- Info ---")
test_df.info()
print("\n--- Description (Numerical Columns) ---")
print(test_df.describe())
print("\n--- Missing Values ---")
print(test_df.isnull().sum())
print("\n--- Duplicate Rows ---")
print(test_df.duplicated().sum())

# --- SAMPLE SUBMISSION INSPECTION ---
print("\n=== SAMPLE SUBMISSION ===\n")
print("--- Info ---")
sample_submission_df.info()
print("\n--- Head ---")
print(sample_submission_df.head())



=== TRAINING DATA ===

--- Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   id               750000 non-null  int64 
 1   Temparature      750000 non-null  int64 
 2   Humidity         750000 non-null  int64 
 3   Moisture         750000 non-null  int64 
 4   Soil Type        750000 non-null  object
 5   Crop Type        750000 non-null  object
 6   Nitrogen         750000 non-null  int64 
 7   Potassium        750000 non-null  int64 
 8   Phosphorous      750000 non-null  int64 
 9   Fertilizer Name  750000 non-null  object
dtypes: int64(7), object(3)
memory usage: 57.2+ MB

--- Description (Numerical Columns) ---
                  id    Temparature       Humidity       Moisture  \
count  750000.000000  750000.000000  750000.000000  750000.000000   
mean   374999.500000      31.503565      61.038912      45.184147   
s

### EDA

In [52]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np

# Separate features and target
X_train = train_df.drop(['Fertilizer Name', 'id'], axis=1)
y_train = train_df['Fertilizer Name']
X_test = test_df.drop('id', axis=1)

# Concatenate for consistent preprocessing
combined = pd.concat([X_train, X_test], axis=0, ignore_index=True)

# Label encode categorical features
le_soil = LabelEncoder()
le_crop = LabelEncoder()
combined['Soil Type'] = le_soil.fit_transform(combined['Soil Type'])
combined['Crop Type'] = le_crop.fit_transform(combined['Crop Type'])

# Scale numerical features
num_cols = ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous']
scaler = StandardScaler()
combined[num_cols] = scaler.fit_transform(combined[num_cols])

# Split back into train and test
X_train_processed = combined.iloc[:len(X_train), :].reset_index(drop=True)
X_test_processed = combined.iloc[len(X_train):, :].reset_index(drop=True)

# Encode target
le_fert = LabelEncoder()
y_train_encoded = le_fert.fit_transform(y_train)


#### for further saving of encoder

In [53]:
import joblib
joblib.dump(le_soil, 'le_soil.pkl')
joblib.dump(le_crop, 'le_crop.pkl')
joblib.dump(le_fert, 'le_fert.pkl')
joblib.dump(scaler, 'scaler.pkl')


['scaler.pkl']

In [54]:
print("Processed training features shape:", X_train_processed.shape)
print("Processed test features shape:", X_test_processed.shape)
print("Encoded target shape:", y_train_encoded.shape)
print("Unique classes in target:", np.unique(y_train_encoded))


Processed training features shape: (750000, 8)
Processed test features shape: (250000, 8)
Encoded target shape: (750000,)
Unique classes in target: [0 1 2 3 4 5 6]


### model building--> random forest.

In [55]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
import numpy as np


In [56]:
def map_at_3(y_true, y_pred_proba):
    # Get indices of top 3 predictions for each sample
    top_3 = np.argsort(y_pred_proba, axis=1)[:, -3:][:, ::-1]
    y_true = np.array(y_true)
    score = 0.0
    for i in range(len(y_true)):
        if y_true[i] in top_3[i]:
            rank = np.where(top_3[i] == y_true[i])[0][0] + 1  # rank (1-based)
            score += 1.0 / rank
    return score / len(y_true)


In [57]:
# Set up cross-validation
n_splits = 2  # You can increase this for more robust results
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

fold_accuracies = []
fold_map3_scores = []

for fold, (train_idx, valid_idx) in enumerate(skf.split(X_train_processed, y_train_encoded)):
    X_tr, X_val = X_train_processed.iloc[train_idx], X_train_processed.iloc[valid_idx]
    y_tr, y_val = y_train_encoded[train_idx], y_train_encoded[valid_idx]

    # Initialize and train the model
    model = RandomForestClassifier(n_estimators=10, random_state=42, n_jobs=-1)
    model.fit(X_tr, y_tr)

    # Predict probabilities for validation set
    y_pred_proba = model.predict_proba(X_val)

    # Calculate accuracy and MAP@3
    accuracy = model.score(X_val, y_val)
    map3 = map_at_3(y_val, y_pred_proba)

    fold_accuracies.append(accuracy)
    fold_map3_scores.append(map3)

    print(f"Fold {fold+1}: Accuracy = {accuracy:.4f}, MAP@3 = {map3:.4f}")

print(f"Average Accuracy: {np.mean(fold_accuracies):.4f}")
print(f"Average MAP@3: {np.mean(fold_map3_scores):.4f}")


Fold 1: Accuracy = 0.1563, MAP@3 = 0.2734
Fold 2: Accuracy = 0.1574, MAP@3 = 0.2743
Average Accuracy: 0.1568
Average MAP@3: 0.2738


In [58]:
final_model = RandomForestClassifier(n_estimators=10, random_state=42, n_jobs=-1)
final_model.fit(X_train_processed, y_train_encoded)


RandomForestClassifier(n_estimators=10, n_jobs=-1, random_state=42)

### predictions and submission file.

In [59]:
# Predict class probabilities for the test set
test_pred_proba = final_model.predict_proba(X_test_processed)


In [60]:
# Get indices of top 3 predictions for each test sample
top_3_indices = np.argsort(test_pred_proba, axis=1)[:, -3:][:, ::-1]

# Convert indices to fertilizer names row by row
predicted_fertilizers = []
for row in top_3_indices:
    fert_names = le_fert.inverse_transform(row)
    predicted_fertilizers.append(' '.join(fert_names))

# Create submission DataFrame
submission = pd.DataFrame({
    'id': test_df['id'],
    'Fertilizer Name': predicted_fertilizers
})

# Save submission file
submission.to_csv('submission.csv', index=False)
print("Submission file created: submission.csv")


Submission file created: submission.csv


In [63]:
!ls
from IPython.display import FileLink
FileLink('submission.csv')


le_crop.pkl  le_fert.pkl  le_soil.pkl  scaler.pkl  submission.csv


/kaggle/working/submission.csv